In [1]:
# Внимание!!! Важно, что бы файлы с данными и исполняемый файл находились в одной папке, 
# тогда пути к тестовым и тренировочным наборам будут содержать только имена файлов.
# 
# В пути к тренировочным и тестовым данным запрежается использовать абсалютную адресацию, 
# то есть адресацию, в которой присутствуют имена папок. Путь должен содержать только имя файла.
#
# Напоминание: под моделью машинного обучения понимаются все действия с исходными данными, 
# которые необходимо произвести, что бы сопоставить признаки целевому значению.

### Область работы 1 (библиотеки)

In [2]:
# Данный блок в области 1 выполняется преподавателем
# 
# данный блок предназначен только для подключения необходимых библиотек
# запрещается подключать библиотеки в других блоках
# запрещается скрывать предупреждения системы
# установка дополнительных библиотек размещается прямо здесь (обязательно закоментированы)
# pip install

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import svm, metrics
import scipy.stats as stats
from scipy.stats import iqr
import sklearn

from sklearn.neighbors import  KNeighborsRegressor
from sklearn.linear_model import LogisticRegression, LinearRegression, Lasso, ElasticNet, Ridge

from sklearn.pipeline import Pipeline, make_pipeline

from sklearn.model_selection import GridSearchCV, KFold, RandomizedSearchCV
from sklearn.model_selection import ShuffleSplit, StratifiedShuffleSplit
from sklearn.model_selection import validation_curve, train_test_split

from sklearn.metrics import make_scorer, mean_absolute_percentage_error
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error
from sklearn.compose import make_column_transformer,ColumnTransformer,TransformedTargetRegressor
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import SimpleImputer, IterativeImputer, KNNImputer

from sklearn.preprocessing import FunctionTransformer, PolynomialFeatures,KBinsDiscretizer
from sklearn.preprocessing import MinMaxScaler,Normalizer, StandardScaler, RobustScaler
from sklearn.preprocessing import OneHotEncoder,OrdinalEncoder, QuantileTransformer, PowerTransformer

from sklearn.preprocessing import KBinsDiscretizer
import category_encoders as ce


### Область работы 2 (выполнение лучшей модели)

In [4]:
# Данный блок(и) в области 2 выполняется преподавателем
#
# В области находится одна, единственная, итоговая модель машинного обучения с однозначными, 
# зафиксированными параметрами
#
# В данной области категорически запрещается искать, выбирать, улучшать, оптимизировать, 
# тюниговать и т.д. модель машинного обучения

In [5]:
# Путь к тренировочному набору
path_train = 'train.csv' # содержит только имя файла, без имен папок
# Путь к тестовому набору
path_test  = 'test.csv' # содержит только имя файла, без имен папок

df_train = pd.read_csv('train.csv')
X_test = pd.read_csv('test.csv')
print(df_train.shape)
print(X_test.shape)

(67598, 12)
(22453, 11)


In [6]:
cat_features = ['color','clarity','cut','symmetry','polish']
axis_features = ['meas_length','meas_width','meas_depth']
nonaxis_num_features = ['size','depth_percent','table_percent']
num_features = ['size','depth_percent','table_percent','meas_length','meas_width','meas_depth']

In [7]:
# уберем выбросы
df_num = df_train[num_features].copy()
df_num.shape

Q1 = df_num.quantile(q=.25)
Q3 = df_num.quantile(q=.75)
IQR = df_num.apply(stats.iqr)
df_iqr = df_num[~((df_num < (Q1-1.5*IQR)) | (df_num > (Q3+1.5*IQR))).any(axis=1)]

mask = [True if index in df_iqr.index else False for index in df_train.index]
df_iqred = df_train[mask].copy()
print(df_iqred.shape)

(56987, 12)


In [8]:
#уберем полные дубликаты 
df_iqred.drop_duplicates(inplace=True)
df = df_iqred.copy()
print(df.shape)


(56838, 12)


In [9]:
X_train = df.drop(columns = 'total_sales_price')
y_train = df.total_sales_price

In [10]:
# Блок(и) обучения и поверки модели

In [11]:
cat_features = ['cut', 'symmetry', 'polish', 'color', 'clarity']
axis_features = ['meas_length', 'meas_width', 'meas_depth', 'size']
num_features = ['depth_percent', 'table_percent']

In [12]:
def binarization(X):
    return np.where(X == 'Excellent', 1, 0)
def weird_gem(df):
    sz = (df['meas_width']**2)*df['meas_depth']*0.0061
    cond = ((abs(sz / df['size'])) - 1) > 0.15
    df.loc[cond, 'meas_length'] = 0
    return df


In [13]:

axis_transformer = Pipeline(steps=[
    ('strange', FunctionTransformer(weird_gem, validate=False)),
    #('missing_num', IterativeImputer(missing_values=0, max_iter=20)),
    ('missing_num', KNNImputer(missing_values=0, n_neighbors=3, weights="uniform")),
    #('bin', KBinsDiscretizer(n_bins=10, encode='ordinal', strategy='uniform')), с ним хуже..
    ('polynom', PolynomialFeatures(2, include_bias=False)),
    ('scaler', StandardScaler())
])

num = Pipeline(steps=[
    ('missing_num', IterativeImputer(missing_values=0, max_iter=20)),
    ('scaler', StandardScaler())
])

clarity_map = [{
    'col' : 'clarity',
    'mapping':{'IF':9, 'VVS1':8, 'VVS2':7,
               'VS1':6, 'VS2':5, 'SI1':4, 'SI2':3, 
               'I1':2, 'I2':1, 'I3':0}
}]

clarity_transformer = Pipeline(steps=[
    ('missing_num', SimpleImputer(strategy = 'most_frequent')),
    ('ce',ce.OrdinalEncoder(mapping=clarity_map)),
    ('scaler', MinMaxScaler())
])    
    
CT = ColumnTransformer([
        ("axis_transformer", axis_transformer, axis_features),
        ("num", num, num_features),
        ("cat", OneHotEncoder(sparse_output=False).set_output(transform="pandas"), cat_features),
        ("ordinal_map", clarity_transformer, ['clarity'])
        ]).set_output(transform='pandas')

display(CT)

ct = CT.fit_transform(X_train)


C:\Users\fueir\anaconda3\lib\site-packages\sklearn\preprocessing\_function_transformer.py:340: UserWarning: With transform="pandas", `func` should return a DataFrame to follow the set_output API.
  warnings.warn(


ColumnTransformer(transformers=[('axis_transformer',
                                 Pipeline(steps=[('strange',
                                                  FunctionTransformer(func=<function weird_gem at 0x000001EC55014040>)),
                                                 ('missing_num',
                                                  KNNImputer(missing_values=0,
                                                             n_neighbors=3)),
                                                 ('polynom',
                                                  PolynomialFeatures(include_bias=False)),
                                                 ('scaler', StandardScaler())]),
                                 ['meas_length', 'meas_width', 'meas_depth',
                                  'size']),
                                ('num',
                                 Pipelin...
                                ('cat', OneHotEncoder(sparse_output=False),
                                 ['cut', 'symmetry', 'polish', 'color',
                                  'clarity']),
                                ('ordinal_map',
                                 Pipeline(steps=[('missing_num',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('ce',
                                                  OrdinalEncoder(mapping=[{'col': 'clarity',
                                                                           'data_type': dtype('O'),
                                                                           'mapping': IF      9
VVS1    8
VVS2    7
VS1     6
VS2     5
SI1     4
SI2     3
I1      2
I2      1
I3      0
dtype: int64}])),
                                                 ('scaler', MinMaxScaler())]),
                                 ['clarity'])])

In [14]:
model = TransformedTargetRegressor(regressor = Pipeline(steps = [
                                                      ('preprocessing', CT),
                                                      ('estimator', KNeighborsRegressor(n_neighbors=4, weights='distance'))]), 
                                         transformer=PowerTransformer(method='box-cox'))
model.fit(X_train, y_train)

TransformedTargetRegressor(regressor=Pipeline(steps=[('preprocessing',
                                                      ColumnTransformer(transformers=[('axis_transformer',
                                                                                       Pipeline(steps=[('strange',
                                                                                                        FunctionTransformer(func=<function weird_gem at 0x000001EC55014040>)),
                                                                                                       ('missing_num',
                                                                                                        KNNImputer(missing_values=0,
                                                                                                                   n_neighbors=3)),
                                                                                                       ('polynom',
                                                                                                        PolynomialFeatures(include_bias=False)),
                                                                                                       ('scaler',
                                                                                                        StandardScaler...
                                                                                       Pipeline(steps=[('missing_num',
                                                                                                        SimpleImputer(strategy='most_frequent')),
                                                                                                       ('ce',
                                                                                                        OrdinalEncoder(mapping=[{'col': 'clarity',
                                                                                                                                 'data_type': dtype('O'),
                                                                                                                                 'mapping': IF      9
VVS1    8
VVS2    7
VS1     6
VS2     5
SI1     4
SI2     3
I1      2
I2      1
I3      0
dtype: int64}])),
                                                                                                       ('scaler',
                                                                                                        MinMaxScaler())]),
                                                                                       ['clarity'])])),
                                                     ('estimator',
                                                      KNeighborsRegressor(n_neighbors=4,
                                                                          weights='distance'))]),
                           transformer=PowerTransformer(method='box-cox'))

In [ ]:
y_pred = model.predict(X_train)
print(f'train MAPE: {mean_absolute_percentage_error(y_train, y_pred)}')


In [ ]:
# Блок предсказания с использованием тестового набора

In [ ]:
y_pred_test = model.predict(X_test)
print(y_pred_test.shape)


In [ ]:
# Название вектора предсказанных значений  y_predict полученого на основании тестового набора
y_predict = y_pred_test
y_predict